# Import the necessary libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
import gensim

%matplotlib inline

In [2]:
#preprocessings a gensim thing

In [3]:
#model = gensim.models.Word2Vec (documents, size=200, window=10, min_count=1, workers=10)
#model.train(documents,total_examples=len(documents),epochs=10)

In [4]:
### Load the data into Pandas dataframe

In [35]:
df = pd.read_excel('NoBS_cd_corrected.xlsx')
df.head()
df1 = pd.read_csv('Datenschutzerklärung.csv',delimiter=';')

ParserError: Error tokenizing data. C error: Expected 2 fields in line 126, saw 3


### Datawrangling
Drop the columns that are not required for the neural network.

In [6]:
#NaN Values ?
#df.fillna(0) replace all NaN with 0

1. sharing data, for business
2. sharing data, beyond business
3. data location, easy to find out
4. data location, hard to find out
5. duration of data storage, not available ???????
6. dpo contact, mail or phone number given
7. dpo contact, hard
8. deletion of data ???????

In [7]:
#making a function to reverse a String by the number
# input int, output string
def meaning(s):
    if s == 1:
        s1 = "sharing data, for business"
    if s == 2:
        s1 = "sharing data, beyond business"
    if s == 3:
        s1 = "data location, easy to find out"
    if s == 4:
        s1 = "data location, hard to find out"
    if s == 5:
        s1 = "duration of data storage, not available"
    if s == 6:
        s1 = "dpo contact, mail or phone number given"
    if s == 7:
        s1 = "dpo contact, hard"
    if s == 8:
        s1 = "deletion of data"
    else:
        s1= "unknown input value"
    return s1

Understand the distribution better.

In [8]:
#sns.countplot(df.v1)
#plt.xlabel('Label')
#plt.title('Number of ham and spam messages')

AttributeError: 'DataFrame' object has no attribute 'v1'

* Create input and output vectors.
* Process the labels.

In [13]:
X = df.text
Y = df.value
le = LabelEncoder()
#Y = le.fit_transform(Y)
#Y = Y.reshape(-1,1)

Split into training and test data.

In [14]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.15)

### Process the data
* Tokenize the data and convert the text to sequences.
* Add padding to ensure that all the sequences have the same shape.
* There are many ways of taking the *max_len* and here an arbitrary length of 150 is chosen.

In [15]:
max_words = 10000 # !!!!!!!!! check if all paragraphs are max 10000 words
max_len = 150
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

In [21]:
df = df.fillna(0)
df

,value,text
0,0,1. When you give it to us or give us permissio...
1,2,"If you connect your Facebook, Google (opens in..."
2,0,2. We also get technical information when you ...
3,0,"Log data. When you use Pinterest, our servers ..."
4,0,Cookie data. We also use “cookies” (small text...
...,...,...
299,4,What if I Access the Sites from Outside of the...
300,0,How Will I Be Informed About Changes Regarding...
301,0,Arbitration; Claims Must be Brought on an Indi...
302,0,"In arbitration, the parties may conduct reason..."


In [23]:
# load the whole embedding into memory
filename = "privacyPolicy.txt"
embeddings_index = dict()
f = open(filename)
for line in f:
	values = line.split()
	word = values[0]
	coefs = array(values[1:], dtype='float64')
	embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

NameError: name 'array' is not defined

vocab_size = 2000
# create a weight matrix for words in training docs
embedding_matrix = np.zeros((vocab_size, 100))
for word, i in tok.word_index.items():
	embedding_vector = embeddings_index.get(word)
	if embedding_vector is not None:
		embedding_matrix[i] = embedding_vector

In [ ]:
model = RNN()
model.summary()
model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])

Fit on the training data.

In [ ]:
model.fit(sequences_matrix,Y_train,batch_size=128,epochs=10,
          validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])

The model performs well on the validation set and this configuration is chosen as the final model.

Process the test set data.

In [ ]:
test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)

Evaluate the model on the test set.

In [ ]:
accr = model.evaluate(test_sequences_matrix,Y_test)

In [ ]:
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))